In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
matplotlib.rcParams['figure.figsize'] = (12, 8) # Adjusts the configuration of the plot we will create

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### A project to find factors with high correlation to gross movies' revenue

### Loading the Data

In [47]:
df = pd.read_csv('../input/movies/movies.csv')
df.head()

# Data Preparation

In [48]:
# Check if there are null values
df.info()

In [49]:
# Find if there are columns with null values
for col in df.columns:
    pct_missing = np.sum(df[col].isnull())
    print('{} - {}'.format(col, pct_missing))

In [50]:
# Check which columns have null values
nan_cols = [i for i in df.columns if df[i].isnull().any()]
nan_cols

In [51]:
# Drop null values
df.dropna(inplace=True)
df.info()

In [52]:
# Change datatype in budget and gross columns to integers
df['budget'] = df.budget.astype('int64')
df['gross'] = df.gross.astype('int64')
df.info()

In [53]:
df.drop(['country','year'], axis=1, inplace=True)
df.head()

In [54]:
#df[['released', 'country']] = df['released'].str.extract('()', expand=True)
df[['released','country']] = df['released'].str.split('(', expand=True)
df['country'] = df['country'].str.replace('[)]',"", )
df['released'] = pd.to_datetime(df.released)
df['year'] = df.released.dt.year
df.head(3)

In [55]:
# To check if the country column has been cleaned of ')'
df['country'].unique()

In [56]:
#Order data in ascending order by budget
df.sort_values(by=['gross'], ascending=False).head(5)

In [57]:
#To remove seemingly dotted lines
#pd.set_option('display.max_rows', None)

In [58]:
#Drop any duplicate in a column, can pick any column
df['company'].drop_duplicates().sort_values(ascending=False).head(5)


In [59]:
# Drop duplicates in the entire dataframe
df.drop_duplicates().head()

## Checking Correlation
#### Hypothesis
- Budget high correlation
- Company correlation

In [60]:
# Scatter plot for Budget and Revenue
plt.scatter(x=df['budget'], y=df['gross'])
plt.title('Effect of Budget on Gross Revenue')
plt.xlabel('Budget')
plt.ylabel('Gross Revenue')

In [61]:
# Plot budget Vs gross using seaborn
sns.regplot(x='budget', y='gross', data=df, scatter_kws={"color":"red"}, line_kws={"color":"blue"})

In [62]:
# 3 Types of correlation 
    #pearson
    #kendall
    #spearman
#We are interested in pearson correlation
df.corr(method='pearson')

In [63]:
correlation_matrix = df.corr(method='pearson')
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix for Numeri Features')
plt.xlabel('Movie Features')
plt.ylabel('Movie Features')

In [64]:
numerized_df = df
for col_name in numerized_df.columns:
    if (numerized_df[col_name].dtype=='object'):
        numerized_df[col_name] = numerized_df[col_name].astype('category')
        numerized_df[col_name] = numerized_df[col_name].cat.codes

numerized_df.head()

In [65]:
correlation_matrix = numerized_df.corr(method='pearson')
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix for Numerical Features')
plt.xlabel('Movie Features')
plt.ylabel('Movie Features')

In [66]:
correlation_mat = numerized_df.corr()

corr_pairs = correlation_mat.unstack()

corr_pairs

In [67]:
sorted_pairs = corr_pairs.sort_values()
sorted_pairs

In [68]:
high_corr = sorted_pairs[(sorted_pairs) > 0.5]
high_corr = high_corr[(high_corr) < 1]
high_corr

There is high correlation between votes and budget with gross revenue.

There is small correlation between company and gross revenue.